In [27]:
import pandas as pd
import numpy as np
import json

NUMBER_PROCESSES = 100

In [110]:
games = pd.read_pickle("../game_data_cleaner/game_dfs_dirty/games.pkl")
ratings_totals = pd.DataFrame(games["BGGId"])
ratings_totals["RatingsPages"] = np.ceil(games["NumUserRatings"].astype(int) / 100).astype("int")
ratings_totals = ratings_totals.sort_values(
    "RatingsPages", ascending=False
).reset_index(drop=True)

total_ratings = ratings_totals["RatingsPages"].sum()
max_ratings_pages = int(np.ceil(total_ratings/NUMBER_PROCESSES))

In [111]:
df_groups = {}
group_counter = 1
position = 0

while len(ratings_totals) > 0:

    group_positions = []
    chunk_size = 0
    bgg_id_lists = []
    ratings_lists = []

    while max_ratings_pages > chunk_size:
        try:
            num_ratings = ratings_totals.iloc[position]['RatingsPages']
            bgg_id = ratings_totals.iloc[position]['BGGId']
            bgg_id_lists.append(bgg_id)
            ratings_lists.append(num_ratings)
            chunk_size += num_ratings
            ratings_totals = ratings_totals.drop(position)
            position += 1

        except:
            break

    # print(chunk_size)
    if len(bgg_id_lists) == 0:
        break
    # group_positions = pd.DataFrame({'BGGId':bgg_id_lists, 'RatingsPages':ratings_lists})
    df_groups[f"group{group_counter}"] = [bgg_id_lists, ratings_lists]
    print(f"group{group_counter} Complete")
    group_counter += 1

group1 Complete
group2 Complete
group3 Complete
group4 Complete
group5 Complete
group6 Complete
group7 Complete
group8 Complete
group9 Complete
group10 Complete
group11 Complete
group12 Complete
group13 Complete
group14 Complete
group15 Complete
group16 Complete
group17 Complete
group18 Complete
group19 Complete
group20 Complete
group21 Complete
group22 Complete
group23 Complete
group24 Complete
group25 Complete
group26 Complete
group27 Complete
group28 Complete
group29 Complete
group30 Complete
group31 Complete
group32 Complete
group33 Complete
group34 Complete
group35 Complete
group36 Complete
group37 Complete
group38 Complete
group39 Complete
group40 Complete
group41 Complete
group42 Complete
group43 Complete
group44 Complete
group45 Complete
group46 Complete
group47 Complete
group48 Complete
group49 Complete


In [113]:
def generate_ratings_urls(game_entries):
    urls_list = []
    bgg_ids = game_entries[0]
    ratings_pages = game_entries[1]

    for i in list(range(0, len(bgg_ids))):
        current_bgg_id = bgg_ids[i]
        max_page_number = ratings_pages[i]
        page_numbers = range(1, max_page_number + 1)
        # print(f"Df last page: {max_page_number}\n", f"First page: {page_numbers[0]}\n", f"Last page: {page_numbers[-1]}")

        for page_number in page_numbers:
            path = f"https://www.boardgamegeek.com/xmlapi2/thing?id={current_bgg_id}&ratingcomments=1&page={str(page_number)}&pagesize=100"
            urls_list.append(path)
    print("\n")

    return urls_list

In [114]:
urls_list = generate_ratings_urls(df_groups['group1'])

In [ ]:
group_urls = {}
group_num = 0

for group, game_entries in df_groups.items():
    group_num += 1
    group_urls["group" + str(group_num)] = generate_ratings_urls(game_entries)

with open("../game_user_scraper/scraper_urls_raw/scraper_urls_ratings.json", "w") as convert_file:
    convert_file.write(json.dumps(group_urls))